In [7]:
from openml import tasks, runs
from sklearn.metrics import accuracy_score

In [8]:
missing_sets = ['analcatdata_asbestos', 'analcatdata_boxing1', 'analcatdata_broadwaymult', 'analcatdata_germangss', 'analcatdata_lawsuit', 'ar4', 'autos', 'baseball', 'bodyfat', 'braziltourism', 'chatfield_4', 'chscase_vine1', 'cloud', 'diabetes', 'diggle_table_a2', 'disclosure_z', 'elusage', 'fri_c0_250_5', 'kc3', 'kidney', 'labor', 'lowbwt', 'lupus',
                'meta', 'mfeat-karhunen', 'mfeat-morphological', 'newton_hema', 'no2', 'plasma_retinol', 'pm10', 'prnn_synth', 'rabe_131', 'rmftsa_sleepdata', 'schizo', 'schlvote', 'sleuth_case2002', 'socmob', 'solar-flare', 'squash-stored', 'squash-unstored', 'tae', 'teachingAssistant', 'transplant', 'triazines', 'veteran', 'visualizing_livestock', 'vote', 'white-clover']

In [9]:
missing_ids = [
    3550, 3540, 3824, 3887, 3542, 3911, 9, 2077, 3644, 2078, 3685, 3680, 3753, 37, 3683, 3794, 3655, 3642, 3915, 3808, 4, 3804, 3562, 3623, 16, 18, 3649, 3749, 3778, 3616, 3555, 3788, 3607, 3557, 3713, 3765, 3797, 2068, 3835, 3848, 47, 3949, 3748, 3653, 3585, 3731, 55, 3872
]

In [ ]:
import json
with open("../turbo_ml/meta_learning/meta_model/algorthm_families.json", "r") as f:
    algorithm_families = json.load(f)
inv_map = {}
families = algorithm_families.keys()
for k, v in algorithm_families.items():
    for i in v:
        inv_map[i] = k
families

dict_keys(['Discriminant_Analysis_(DA)', 'Bayesian_Methods_(BY)', 'Neural_Networks_(NNET)', 'Support_Vector_Machines_(SVM)', 'Decision_Trees_(DT)', 'Rule-Based_Methods_(RL)', 'Boosting_(BST)', 'Bagging_(BAG)', 'Stacking_(STC)', 'Random_Forests_(RF)', 'Other_Ensembles_(OEN)', 'Generalized_Linear_Models_(GLM)', 'Nearest_Neighbor_Methods_(NN)', 'Partial_Least_Squares_and_Principal_Component_Regression_(PLSR)', 'Logistic_and_Multinomial_Regression_(LMR)', 'Multivariate_Adaptive_Regression_Splines_(MARS)', 'Other_Methods_(OM)'])

In [11]:
scores = {name: {family: [] for family in families} for name in missing_sets}

In [12]:
import random
import re
for id, name in zip(missing_ids, missing_sets):
    print(name)
    runs_df = runs.list_runs(task=[id], output_format='dataframe')
    run_ids = runs_df['run_id'].tolist()
    random.shuffle(run_ids)
    for run_id in run_ids[:100]:
        run = runs.get_run(run_id)
        desc = run.__str__()
        try:
            metric = re.search(
                r'Metric.{10}: ([^\s]+)', desc).group(0).split(': ')[-1]
            flow_name = re.search(
                r'Flow Name[.]+: ([^\s]+)', desc).group(0).split(': ')[-1].split('(')[0]
            result = re.search(
                r'Result.{10}: 0.[0-9]+', desc).group(0).split(': ')[-1]
        except AttributeError:
            continue
        if metric != 'predictive_accuracy':
            continue
        if flow_name[:5] == 'weka.':
            flow_name = ('_').join([flow_name[5:], 'w'])

        if flow_name in inv_map:
            family = inv_map[flow_name]
        else:
            if "bagging" in flow_name.lower():
                family = "Bagging_(BAG)"
            elif "boost" in flow_name.lower():
                family = "Boosting_(BST)"
            elif "bayes" in flow_name.lower():
                family = "Bayesian_Methods_(BY)"
            elif "nn" in flow_name.lower():
                family = "Neural_Networks_(NNET)"
            elif "svm" in flow_name.lower():
                family = "Support_Vector_Machines_(SVM)"
            elif "logistic" in flow_name.lower():
                family = "Logistic_and_Multinomial_Regression_(LMR)"
            elif "forest" in flow_name.lower():
                family = "Random_Forests_(RF)"
            else:
                print(flow_name)
                continue
        # print(family, flow_name, result)
        scores[name][family].append(float(result))
    for family in families:
        if len(scores[name][family]) == 0:
            scores[name][family] = 0
        else:
            scores[name][family] = max(scores[name][family])

analcatdata_asbestos
AttributeSelectedClassifier_InfoGainAttributeEval_Ranker_IBk_w
LADTree_w
FilteredClassifier_Discretize_J48_w
kf.AttributeSelection-Ranker-InfoGain-SMO_w
AttributeSelectedClassifier_CfsSubsetEval_BestFirst_IBk_w
A1DE_w
mlr.classif.develpartykit.ctree
AttributeSelectedClassifier_GainRatioAttributeEval_Ranker_HoeffdingTree_w
A1DE_w
A1DE_w
A1DE_w
FilteredClassifier_Discretize_J48_w
ADTree_w
FilteredClassifier_Discretize_J48_w
A1DE_w
AttributeSelectedClassifier_GainRatioAttributeEval_Ranker_J48_w
classif.lda
kf.AttributeSelection-BestFirst-CfsSubsetEval-KStar_w
AttributeSelectedClassifier_InfoGainAttributeEval_Ranker_MultilayerPerceptron_w
mlr.classif.develpartykit.ctree
AttributeSelectedClassifier_CfsSubsetEval_BestFirst_SGD_w
classif.lda
AttributeSelectedClassifier_InfoGainAttributeEval_Ranker_HoeffdingTree_w
A1DE_w
ADTree_w
mlr.classif.develpartykit.ctree
A1DE_w
A1DE_w
FilteredClassifier_Discretize_J48_w
AttributeSelectedClassifier_PrincipalComponents_Ranker_J48_w
kf

In [13]:
scores

{'analcatdata_asbestos': {'Discriminant_Analysis_(DA)': 0,
  'Bayesian_Methods_(BY)': 0.783133,
  'Neural_Networks_(NNET)': 0.807229,
  'Support_Vector_Machines_(SVM)': 0.722892,
  'Decision_Trees_(DT)': 0.73494,
  'Rule-Based_Methods_(RL)': 0.783133,
  'Boosting_(BST)': 0.807229,
  'Bagging_(BAG)': 0.783133,
  'Stacking_(STC)': 0,
  'Random_Forests_(RF)': 0.783133,
  'Other_Ensembles_(OEN)': 0.759036,
  'Generalized_Linear_Models_(GLM)': 0,
  'Nearest_Neighbor_Methods_(NN)': 0.783133,
  'Partial_Least_Squares_and_Principal_Component_Regression_(PLSR)': 0,
  'Logistic_and_Multinomial_Regression_(LMR)': 0.795181,
  'Multivariate_Adaptive_Regression_Splines_(MARS)': 0,
  'Other_Methods_(OM)': 0.771084},
 'analcatdata_boxing1': {'Discriminant_Analysis_(DA)': 0,
  'Bayesian_Methods_(BY)': 0.833333,
  'Neural_Networks_(NNET)': 0.791667,
  'Support_Vector_Machines_(SVM)': 0.825,
  'Decision_Trees_(DT)': 0.841667,
  'Rule-Based_Methods_(RL)': 0.816667,
  'Boosting_(BST)': 0.85,
  'Bagging_(BA

In [24]:
import pandas as pd


df = pd.DataFrame.from_dict(scores)

df = df.T
df.reset_index(inplace=True)
df.rename(columns={'index': 'name'}, inplace=True)
df.set_index('name', inplace=True)
df.to_csv("../data/missing_family_scores.csv")